# API idealista

Find a list of sqm of houses/flats in Barcelona from the API of Idealista https://api.idealista.com/

## 0. Import libraries

In [1]:
import urllib 
import requests
import json
import pandas as pd
# from pandas.io.json import json_normalize
import os
# pprint --> provides a capability to “pretty-print” arbitrary Python data structures in a form which can be used as input to the interpreter
# from pprint import pprint
import base64

## 1. API credentials

In [9]:
# Sent by idealista
apikey_id='p2pmrhbz0kapkq6x43h15g7hs4i55jbh' 
secret_id='kAWoimO7YSFk' 

In [13]:
apikey = apikey_id.encode(encoding='UTF-8')
secret = secret_id.encode(encoding='UTF-8')
type(apikey)

bytes

In [4]:
# Access is free to a maximum of 100 req/month and it’s limited by 1 req/sec. 

In [5]:
# Http() = 'https://www.idealista.com/'

## 2. OAuth 2.0 Authentication

In [14]:
# Request a Bearer token for OAuth authentication.

def get_oauth_token():
    # http_obj = Http()
    
    # Resource URL
    url = "https://api.idealista.com/oauth/token"
    
    '''
    Parse the API key
    parse.quote_plus -> The URL quoting functions focus on taking program data and making it safe for use as URL components 
    by quoting special characters and appropriately encoding non-ASCII text.
    '''
    apikey= urllib.parse.quote_plus('Provided_API_key') 
    secret= urllib.parse.quote_plus('Provided_API_secret')
    
    # Encoding the authorization header
    auth = base64.b64encode(apikey + ':' + secret)
    
    # Parameters
    body = {'grant_type':'client_credentials'}
    # scope = {'scope':'read'}
    
    # Headers
    headers = {'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8','Authorization' : 'Basic ' + auth}
#     resp, content = http_obj.request(url,method='POST',headers=headers, body=urllib.parse.urlencode(body))
    
    # Response
    # response = http_obj.request(url,method='POST',headers=headers, body=urllib.parse.urlencode(body), scope=scope)
    
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token
#     return response

In [15]:
get_oauth_token()

TypeError: a bytes-like object is required, not 'str'

In [16]:
auth = base64.b64encode(apikey + ':' + secret)

TypeError: can't concat str to bytes

## 3. Property search

In [ ]:
# # Query the API
## Main call for property search purposes

# def search_api(token):
#     http_obj = Http()
#     url = "http://api.idealista.com/3.5/es/search?center=40.42938099999995,-3.7097526269835726&country=es&maxItems=50&numPage=1&distance=452&propertyType=bedrooms&operation=rent"
#     headers = {'Authorization' : 'Bearer ' + token}
#     resp, content = http_obj.request(url,method='POST',headers=headers)
#     return content